In [1]:
import torch
import pickle
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm.notebook import tqdm
import os
from PIL import Image

def dump_data(data, filename):
    print('writing file: ' + filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [2]:
fmri_stim = np.load("/projects/crunchie/boyanova/EEG_Things/eeg_prep/scripts/fmri_train_stim.npy", allow_pickle=True)
main_path_stim = "/projects/crunchie/boyanova/EEG_Things/data_set/Images"

image_paths = []
for im in tqdm(fmri_stim):
    im_cat = im.split(".")[0][0:-4]
    image_paths.append(os.path.join(main_path_stim, im_cat, im))    

  0%|          | 0/8640 [00:00<?, ?it/s]

In [3]:
batch_size = 8 
all_image_embeddings = []

# Process images in batches
for i in tqdm(range(0, len(image_paths), batch_size)):
    # Load and preprocess a batch of images
    batch_images = []
    for path in image_paths[i:i + batch_size]:
        image = Image.open(path).convert("RGB")
        batch_images.append(image)

    # Use the CLIP processor to prepare the batch of images
    inputs = processor(images=batch_images, return_tensors="pt", padding=True)

    # Obtain the image embeddings
    with torch.no_grad():
        image_embeddings = model.get_image_features(**inputs)

    # Detach the embeddings and move to CPU
    all_image_embeddings.append(image_embeddings.detach().cpu().numpy())

# Convert the list of embeddings to a single NumPy array
all_image_embeddings = np.concatenate(all_image_embeddings, axis=0)
print("Total Image Embeddings Shape:", all_image_embeddings.shape)


  0%|          | 0/1080 [00:00<?, ?it/s]

Total Image Embeddings Shape: (8640, 768)


In [5]:
CLIP_vis = {"stimuli": fmri_stim,
            "stimuli_paths": image_paths,
            "embeddings": all_image_embeddings}

In [6]:
dump_data(CLIP_vis, "/projects/crunchie/boyanova/EEG_Things/eeg_prep/scripts/CLIP_vis_fmri_512.pickle")

writing file: /projects/crunchie/boyanova/EEG_Things/eeg_prep/scripts/CLIP_vis_fmri_512.pickle
